<a href="https://colab.research.google.com/github/Dharshan4038/PDF_Q-A_USING_RAG/blob/main/RAG_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# %pip install langchain ipykernel langchain_community pypdf langchain-text-splitters langchain-openai openai easyocr fitz frontend pymupdf faiss-cpu

In [35]:
# !pip install PyMuPDF easyocr pillow langchain

In [36]:
# !pip uninstall -y PyMuPDF
# !pip install PyMuPDF

In [37]:
# !pip list

In [8]:
!pip install pdf2image

In [9]:
import fitz  # PyMuPDF
import easyocr
import io
from PIL import Image
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings  # Updated for embeddings
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_openai import AzureOpenAIEmbeddings

In [19]:
# Extract images from PDF
def extract_image_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        images = page.get_images(full=True)

        for image_num, image in enumerate(images):
            xref = image[0]
            base_image = pdf_document.extract_image(xref)
            img_bytes = base_image["image"]
            img_pil = Image.open(io.BytesIO(img_bytes))
            img_path = f"/content/drive/MyDrive/output/image_{page_num}_{image_num}.png"
            img_pil.save(img_path)
            image_paths.append(img_path)

    return image_paths

In [11]:
# Extract text from images using EasyOCR
def extract_text_from_images_easyocr(image_paths):
    reader = easyocr.Reader(['en'])
    text = ""

    for image_path in image_paths:
        results = reader.readtext(image_path)
        for result in results:
            text += result[1] + "\n"

    return text

In [12]:
# Step 3: Preprocess the text and create a Document
def preprocess_text(text):
    lines = text.split("\n")
    for line in lines:
        print(line)

In [13]:
# Load text into Langchain's Document structure
def load_image_text_as_document(text):
    return Document(page_content=text, metadata={"source": "extracted_images"})

In [14]:
# Split documents into chunks
def split_docs(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return text_splitter.split_documents(data)

In [20]:
# Main flow for processing PDF and images
def process_pdf_and_images(pdf_path):
    # Extract images and text from the PDF
    image_paths = extract_image_from_pdf(pdf_path)
    text_from_images = extract_text_from_images_easyocr(image_paths)

    # Preprocess and create document for image-based text
    preprocess_text(text_from_images)
    img_doc = load_image_text_as_document(text_from_images)

    # Load the PDF content into Langchain's Document format
    pdf_loader = PyPDFLoader(pdf_path)
    pdf_docs = pdf_loader.load()

    # Append the image-based document to the PDF documents
    pdf_docs.append(img_doc)

    # Split all documents into smaller chunks
    split_documents = split_docs(pdf_docs)
    return split_documents

In [21]:
# Embedding and Vector Store creation
def create_faiss_vector_store(documents):
    # Initialize the embedding model
    embedding_model =  AzureOpenAIEmbeddings(
    azure_endpoint="MYENDPOINT",
    api_key="MYKEY",
    model="text-embedding-ada-002",
    azure_deployment="aiml-interns-embedding-ada-002-model",
)

    # Create a FAISS vector store from the processed documents
    vectorstore = FAISS.from_documents(documents, embedding_model)
    return vectorstore

In [22]:
# Full execution
pdf_path = "/content/drive/MyDrive/RAG/attention.pdf"
documents = process_pdf_and_images(pdf_path)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Output
Probabilities
Softmax
Linear
Add & Norm
Feed
Forward
Add & Norm
Add & Norm
Multi-Head
Feed
Attention
Forward
Nx
Add & Norm
Nx
Add & Norm
Masked
Multi-Head
Multi-Head
Attention
Attention
Positional
Positional
Encoding
Encoding
Input
Output
Embedding
Embedding
Inputs
Outputs
(shifted right)
Linear
Concat
Scaled Dot-Product
Attention
Linear
Linear
Linear
K
MatMul
SoftMax
Mask (opt )
Scale
MatMul



In [23]:
vectorstore = create_faiss_vector_store(documents)

In [24]:
from langchain_openai import AzureChatOpenAI
import os
os.environ["AZURE_OPENAI_API_KEY"] = "MYKEY"
os.environ["AZURE_OPENAI_ENDPOINT"] = "MYENDPOINT"

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [26]:
from langchain.vectorstores import  FAISS
from langchain.prompts import ChatPromptTemplate
template = """You are an AI assistant for question-answering tasks.
        Use the following pieces of context to answer the question. If you don't know the
        answer, please output 'No answer'.Use Maximum ten sentence and keep the answer concise
        Question: {question} Context: {context}
        """

In [27]:
prompt = ChatPromptTemplate.from_template(template)
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.runnable import  RunnablePassthrough
from langchain.schema.output_parser import  StrOutputParser

In [28]:
output_parser = StrOutputParser()

In [29]:
llm = AzureChatOpenAI(
    azure_endpoint="MYENDPOINT",
    api_key="MYKEY",
    model = "gpt-35-turbo",
    max_tokens=50,
    azure_deployment="GPT-35-TURBO",
    api_version="2024-06-01"
)

<ipython-input-29-263f8acba241>:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  llm = AzureChatOpenAI(


In [30]:
retriever = vectorstore.as_retriever()

In [31]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
# Create a question-answering chain (QA Chain) that will use the LLM
qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

# Define the ConversationalRetrievalChain to structure the pipeline
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

<ipython-input-31-8f3fdf5ae8f3>:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)


In [32]:
# Example question
question = "What is the BLEU Score of EN-DE of model ConvS2S Ensemble?"

# Initialize an empty chat history
chat_history = []

# Invoke the chain
try:
    response = chain({"question": question, "chat_history": chat_history})
    answer = response["answer"]
    source_documents = response["source_documents"]

    print("Answer:", answer)
    print("Source Documents:", source_documents)
except Exception as e:
    print(f"An error occurred: {e}")

<ipython-input-32-869585f555d2>:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain({"question": question, "chat_history": chat_history})


Answer: The BLEU score of the EN-DE model ConvS2S Ensemble is 26.36.
Source Documents: [Document(metadata={'source': '/content/drive/MyDrive/RAG/attention.pdf', 'page': 7}, page_content='Table 2: The Transformer achieves better BLEU scores than previous state-of-the-art models on the\nEnglish-to-German and English-to-French newstest2014 tests at a fraction of the training cost.\nModelBLEU Training Cost (FLOPs)\nEN-DE EN-FR EN-DE EN-FR\nByteNet [15] 23.75\nDeep-Att + PosUnk [32] 39.2 1.0·1020\nGNMT + RL [31] 24.6 39.92 2.3·10191.4·1020\nConvS2S [8] 25.16 40.46 9.6·10181.5·1020\nMoE [26] 26.03 40.56 2.0·10191.2·1020\nDeep-Att + PosUnk Ensemble [32] 40.4 8.0·1020\nGNMT + RL Ensemble [31] 26.30 41.16 1.8·10201.1·1021\nConvS2S Ensemble [8] 26.36 41.29 7.7·10191.2·1021\nTransformer (base model) 27.3 38.1 3.3·1018\nTransformer (big) 28.4 41.0 2.3·1019\nLabel Smoothing During training, we employed label smoothing of value ϵls= 0.1[30]. This\nhurts perplexity, as the model learns to be more uns

In [33]:
question = "What is the BLEU Score of EN-FR of model Transformer(big)?"

# Initialize an empty chat history
chat_history = []

# Invoke the chain
try:
    response = chain({"question": question, "chat_history": chat_history})
    answer = response["answer"]
    source_documents = response["source_documents"]

    print("Answer:", answer)
    print("Source Documents:", source_documents)
except Exception as e:
    print(f"An error occurred: {e}")

Answer: The BLEU score of the EN-FR translation task for the Transformer (big) model is 41.0.
Source Documents: [Document(metadata={'source': '/content/drive/MyDrive/RAG/attention.pdf', 'page': 7}, page_content='On the WMT 2014 English-to-German translation task, the big transformer model (Transformer (big)\nin Table 2) outperforms the best previously reported models (including ensembles) by more than 2.0\nBLEU, establishing a new state-of-the-art BLEU score of 28.4. The conﬁguration of this model is\nlisted in the bottom line of Table 3. Training took 3.5days on 8P100 GPUs. Even our base model\nsurpasses all previously published models and ensembles, at a fraction of the training cost of any of\nthe competitive models.\nOn the WMT 2014 English-to-French translation task, our big model achieves a BLEU score of 41.0,\noutperforming all of the previously published single models, at less than 1/4the training cost of the\nprevious state-of-the-art model. The Transformer (big) model trained